- we need to check & temperature
- we can get pricing data (Noel)
- we should try to establish the accuracy of current forecast demand vs actual
- temperature data...
- what does AEMO say? residential & commercial demand, temperature, 
- we should start by replicating the AEMO
- spectral analysis reading from william
- 

In [1]:
import pandas as pd
from pandas.plotting import autocorrelation_plot
import os
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
# general settings
class CFG:
    data_path = './../data'
    img_dim1 = 20
    img_dim2 = 10
    

# adjust the parameters for displayed figures
plt.style.use('fivethirtyeight')
plt.rcParams.update({'figure.figsize': (CFG.img_dim1, CFG.img_dim2)})  

## NSW Forecast Demand

In [3]:
nsw_forecastdemand = pd.read_csv(
    os.path.join(CFG.data_path, 'NSW', 'forecastdemand_nsw.csv'), 
    parse_dates=['LASTCHANGED', 'DATETIME']
)

In [4]:
nsw_forecastdemand.head(10)

,PREDISPATCHSEQNO,REGIONID,PERIODID,FORECASTDEMAND,LASTCHANGED,DATETIME
0,2009123018,NSW1,71,7832.04,2009-12-30 12:31:49,2010-01-01
1,2009123019,NSW1,70,7832.04,2009-12-30 13:01:43,2010-01-01
2,2009123020,NSW1,69,7832.03,2009-12-30 13:31:36,2010-01-01
3,2009123021,NSW1,68,7832.03,2009-12-30 14:01:44,2010-01-01
4,2009123022,NSW1,67,7830.96,2009-12-30 14:31:35,2010-01-01
5,2009123023,NSW1,66,7842.03,2009-12-30 15:01:42,2010-01-01
6,2009123024,NSW1,65,7841.08,2009-12-30 15:31:50,2010-01-01
7,2009123025,NSW1,64,7842.07,2009-12-30 16:01:43,2010-01-01
8,2009123026,NSW1,63,7841.90,2009-12-30 16:31:34,2010-01-01
9,2009123027,NSW1,62,7841.90,2009-12-30 17:01:39,2010-01-01


In [5]:
nsw_forecastdemand.tail()

,PREDISPATCHSEQNO,REGIONID,PERIODID,FORECASTDEMAND,LASTCHANGED,DATETIME
10906014,2021031736,NSW1,5,7028.45,2021-03-17 21:31:31,2021-03-18
10906015,2021031737,NSW1,4,7041.65,2021-03-17 22:01:34,2021-03-18
10906016,2021031738,NSW1,3,7087.60,2021-03-17 22:31:36,2021-03-18
10906017,2021031739,NSW1,2,7087.28,2021-03-17 23:01:36,2021-03-18
10906018,2021031740,NSW1,1,7126.37,2021-03-17 23:31:33,2021-03-18


In [6]:
nsw_forecastdemand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10906019 entries, 0 to 10906018
Data columns (total 6 columns):
 #   Column            Dtype         
---  ------            -----         
 0   PREDISPATCHSEQNO  int64         
 1   REGIONID          object        
 2   PERIODID          int64         
 3   FORECASTDEMAND    float64       
 4   LASTCHANGED       datetime64[ns]
 5   DATETIME          datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(2), object(1)
memory usage: 499.2+ MB


In [7]:
nsw_forecastdemand['FORECASTDEMAND'].describe()

count    1.090602e+07
mean     8.066719e+03
std      1.315959e+03
min      4.422460e+03
25%      7.077970e+03
50%      8.007550e+03
75%      8.925610e+03
max      1.473666e+04
Name: FORECASTDEMAND, dtype: float64

In [8]:
nsw_forecastdemand.shape

(10906019, 6)

In [9]:
nsw_forecastdemand.min()

PREDISPATCHSEQNO             2009123018
REGIONID                           NSW1
PERIODID                              1
FORECASTDEMAND                  4422.46
LASTCHANGED         2009-12-30 12:31:49
DATETIME            2010-01-01 00:00:00
dtype: object

In [10]:
nsw_forecastdemand.max()

PREDISPATCHSEQNO             2021031740
REGIONID                           NSW1
PERIODID                             79
FORECASTDEMAND                 14736.66
LASTCHANGED         2021-03-17 23:31:33
DATETIME            2021-03-18 00:00:00
dtype: object

In [11]:
nsw_forecastdemand.isnull().sum()

PREDISPATCHSEQNO    0
REGIONID            0
PERIODID            0
FORECASTDEMAND      0
LASTCHANGED         0
DATETIME            0
dtype: int64

In [12]:
nsw_forecastdemand.isna().sum()

PREDISPATCHSEQNO    0
REGIONID            0
PERIODID            0
FORECASTDEMAND      0
LASTCHANGED         0
DATETIME            0
dtype: int64

In [13]:
nsw_forecastdemand.nunique()

PREDISPATCHSEQNO    196571
REGIONID                 1
PERIODID                79
FORECASTDEMAND      681384
LASTCHANGED         196571
DATETIME            196513
dtype: int64

In [14]:
nsw_forecastdemand.dtypes

PREDISPATCHSEQNO             int64
REGIONID                    object
PERIODID                     int64
FORECASTDEMAND             float64
LASTCHANGED         datetime64[ns]
DATETIME            datetime64[ns]
dtype: object

## NSW temperature

In [15]:
nsw_temperature = pd.read_csv(
    os.path.join(CFG.data_path, 'NSW', 'temperature_nsw.csv'), 
    parse_dates=['DATETIME'], 
    dayfirst=True
)

In [16]:
nsw_temperature['DATE'] = pd.to_datetime(nsw_temperature['DATETIME'])

In [17]:
nsw_temperature.dtypes

LOCATION               object
DATETIME       datetime64[ns]
TEMPERATURE           float64
DATE           datetime64[ns]
dtype: object

In [18]:
nsw_temperature[['DATE', 'TEMPERATURE']].set_index('DATE').resample('30min').mean()

,TEMPERATURE
DATE,
2010-01-01 00:00:00,23.10
2010-01-01 00:30:00,22.80
2010-01-01 01:00:00,22.60
2010-01-01 01:30:00,22.50
2010-01-01 02:00:00,22.50
...,...
2021-03-17 22:00:00,19.70
2021-03-17 22:30:00,19.50
2021-03-17 23:00:00,19.05


In [19]:
# nsw_temperature.groupby(pd.Grouper(key='DATE', freq='30min')).mean().reset_index()

In [20]:
nsw_temperature.head(10)

,LOCATION,DATETIME,TEMPERATURE,DATE
0,Bankstown,2010-01-01 00:00:00,23.1,2010-01-01 00:00:00
1,Bankstown,2010-01-01 00:01:00,23.1,2010-01-01 00:01:00
2,Bankstown,2010-01-01 00:30:00,22.9,2010-01-01 00:30:00
3,Bankstown,2010-01-01 00:50:00,22.7,2010-01-01 00:50:00
4,Bankstown,2010-01-01 01:00:00,22.6,2010-01-01 01:00:00
5,Bankstown,2010-01-01 01:30:00,22.5,2010-01-01 01:30:00
6,Bankstown,2010-01-01 02:00:00,22.5,2010-01-01 02:00:00
7,Bankstown,2010-01-01 02:30:00,22.4,2010-01-01 02:30:00
8,Bankstown,2010-01-01 03:00:00,22.3,2010-01-01 03:00:00
9,Bankstown,2010-01-01 03:30:00,22.3,2010-01-01 03:30:00


In [21]:
nsw_temperature.tail(10)

,LOCATION,DATETIME,TEMPERATURE,DATE
220316,Bankstown,2021-03-17 21:00:00,19.2,2021-03-17 21:00:00
220317,Bankstown,2021-03-17 21:01:00,18.4,2021-03-17 21:01:00
220318,Bankstown,2021-03-17 21:30:00,19.7,2021-03-17 21:30:00
220319,Bankstown,2021-03-17 22:00:00,19.7,2021-03-17 22:00:00
220320,Bankstown,2021-03-17 22:30:00,19.5,2021-03-17 22:30:00
220321,Bankstown,2021-03-17 23:00:00,19.1,2021-03-17 23:00:00
220322,Bankstown,2021-03-17 23:20:00,19.0,2021-03-17 23:20:00
220323,Bankstown,2021-03-17 23:30:00,18.8,2021-03-17 23:30:00
220324,Bankstown,2021-03-17 23:34:00,18.8,2021-03-17 23:34:00
220325,Bankstown,2021-03-18 00:00:00,18.6,2021-03-18 00:00:00


In [22]:
nsw_temperature.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220326 entries, 0 to 220325
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   LOCATION     220326 non-null  object        
 1   DATETIME     220326 non-null  datetime64[ns]
 2   TEMPERATURE  220326 non-null  float64       
 3   DATE         220326 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(1), object(1)
memory usage: 6.7+ MB


In [23]:
nsw_temperature['TEMPERATURE'].describe()

count    220326.000000
mean         17.418827
std           5.849763
min          -1.300000
25%          13.400000
50%          17.700000
75%          21.300000
max          44.700000
Name: TEMPERATURE, dtype: float64

In [24]:
nsw_temperature.shape

(220326, 4)

In [25]:
nsw_temperature.min()

LOCATION                 Bankstown
DATETIME       2010-01-01 00:00:00
TEMPERATURE                   -1.3
DATE           2010-01-01 00:00:00
dtype: object

In [26]:
nsw_temperature.max()

LOCATION                 Bankstown
DATETIME       2021-03-18 00:00:00
TEMPERATURE                   44.7
DATE           2021-03-18 00:00:00
dtype: object

In [27]:
nsw_temperature.isnull().sum()

LOCATION       0
DATETIME       0
TEMPERATURE    0
DATE           0
dtype: int64

In [28]:
nsw_temperature.isna().sum()

LOCATION       0
DATETIME       0
TEMPERATURE    0
DATE           0
dtype: int64

In [29]:
nsw_temperature.nunique()

LOCATION            1
DATETIME       220313
TEMPERATURE       439
DATE           220313
dtype: int64

In [30]:
nsw_temperature.dtypes

LOCATION               object
DATETIME       datetime64[ns]
TEMPERATURE           float64
DATE           datetime64[ns]
dtype: object

## NSW Total Demand

In [31]:
nsw_totaldemand = pd.read_csv(
    os.path.join(CFG.data_path, 'NSW', 'totaldemand_nsw.csv'), 
    parse_dates=['DATETIME'], 
    dayfirst=True
)

In [32]:
nsw_totaldemand.head()

,DATETIME,TOTALDEMAND,REGIONID
0,2010-01-01 00:00:00,8038.00,NSW1
1,2010-01-01 00:30:00,7809.31,NSW1
2,2010-01-01 01:00:00,7483.69,NSW1
3,2010-01-01 01:30:00,7117.23,NSW1
4,2010-01-01 02:00:00,6812.03,NSW1


In [33]:
nsw_totaldemand.tail()

,DATETIME,TOTALDEMAND,REGIONID
196508,2021-03-17 22:00:00,7419.77,NSW1
196509,2021-03-17 22:30:00,7417.91,NSW1
196510,2021-03-17 23:00:00,7287.32,NSW1
196511,2021-03-17 23:30:00,7172.39,NSW1
196512,2021-03-18 00:00:00,7094.51,NSW1


In [34]:
nsw_totaldemand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196513 entries, 0 to 196512
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   DATETIME     196513 non-null  datetime64[ns]
 1   TOTALDEMAND  196513 non-null  float64       
 2   REGIONID     196513 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 4.5+ MB


In [35]:
nsw_totaldemand['TOTALDEMAND'].describe()

count    196513.000000
mean       8113.145859
std        1299.532774
min        5074.630000
25%        7150.070000
50%        8053.230000
75%        8958.550000
max       14579.860000
Name: TOTALDEMAND, dtype: float64

In [36]:
nsw_totaldemand.shape

(196513, 3)

In [37]:
nsw_totaldemand.min()

DATETIME       2010-01-01 00:00:00
TOTALDEMAND                5074.63
REGIONID                      NSW1
dtype: object

In [38]:
nsw_totaldemand.max()

DATETIME       2021-03-18 00:00:00
TOTALDEMAND               14579.86
REGIONID                      NSW1
dtype: object

In [39]:
nsw_totaldemand.isnull().sum()

DATETIME       0
TOTALDEMAND    0
REGIONID       0
dtype: int64

In [40]:
nsw_totaldemand.isna().sum()

DATETIME       0
TOTALDEMAND    0
REGIONID       0
dtype: int64

In [41]:
nsw_totaldemand.nunique()

DATETIME       196513
TOTALDEMAND    160515
REGIONID            1
dtype: int64

In [42]:
nsw_totaldemand.dtypes

DATETIME       datetime64[ns]
TOTALDEMAND           float64
REGIONID               object
dtype: object

## merge step 1

In [43]:
merged_df = pd.merge(
    nsw_totaldemand[['DATETIME', 'TOTALDEMAND']], 
    nsw_forecastdemand[['FORECASTDEMAND', 'DATETIME']], 
    on=['DATETIME'],
    how='inner'
)

# fix_me: need to group by 30min and then merge dataframes together. left inner join is not great. drop location column

In [44]:
merged_df.head(100)

,DATETIME,TOTALDEMAND,FORECASTDEMAND
0,2010-01-01 00:00:00,8038.00,7832.04
1,2010-01-01 00:00:00,8038.00,7832.04
2,2010-01-01 00:00:00,8038.00,7832.03
3,2010-01-01 00:00:00,8038.00,7832.03
4,2010-01-01 00:00:00,8038.00,7830.96
...,...,...,...
95,2010-01-01 00:30:00,7809.31,7715.68
96,2010-01-01 00:30:00,7809.31,7716.52
97,2010-01-01 00:30:00,7809.31,7716.82
98,2010-01-01 00:30:00,7809.31,7709.87


In [45]:
merged_df.isnull().sum()

DATETIME          0
TOTALDEMAND       0
FORECASTDEMAND    0
dtype: int64

## merge step 2

In [46]:
nsw_df = pd.merge(
    merged_df, 
    nsw_temperature[['DATE', 'TEMPERATURE']],
    on='DATE', 
    how='outer'
)

# fix_me: the sampling rate instead of using inner join, which drops about 26k rows.
#  bucketing
#  groupby, fill using mean? or dropna to keep to the 30 min interval? 
#  we need to get special days
#  we need to get the temperature data
#  we need to get the pricing data

KeyError: 'DATE'

## Quick Check of merged df

In [ ]:
nsw_df.isnull().sum()

In [ ]:
nsw_df.head(5)

In [ ]:
nsw_df.tail(5)

In [ ]:
nsw_df.shape

In [ ]:
nsw_df.info()

In [ ]:
# nsw_df.min()

In [ ]:
# nsw_df.max()

In [ ]:
nsw_df.describe()

In [ ]:
nsw_df.nunique()

In [ ]:
nsw_df.dtypes

In [ ]:
nsw_df.isna().sum()

In [ ]:
nsw_df.to_parquet(
    os.path.join(CFG.data_path, 'NSW', 'nsw_df.parquet'),
    engine='pyarrow', 
    compression='snappy'
)

In [ ]:
nsw_df.to_csv(
    os.path.join(CFG.data_path, 'NSW', 'nsw_df.csv'), 
    header=True
)

## 

## Plots

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(nsw_df['DATETIME'], nsw_df['FORECASTDEMAND'], label='Forecast Demand')
plt.plot(nsw_df['DATETIME'], nsw_df['TOTALDEMAND'], label='Actual Demand', alpha=0.7)
plt.xlabel('Datetime')
plt.ylabel('Price')
plt.title('Forecast vs Actual Price Over Time')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(nsw_df['DATETIME'], nsw_df['TEMPERATURE'], color='orange', label='TEMPERATURE')
plt.xlabel('Datetime')
plt.ylabel('Temperature')
plt.title('Temperature Over Time')
plt.legend()
plt.show()

## scatter plot

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(nsw_df['TEMPERATURE'], nsw_df['TOTALDEMAND'], label='Actual Demand', alpha=0.5)
plt.scatter(nsw_df['TEMPERATURE'], nsw_df['FORECASTDEMAND'], label='Forecast Demand', alpha=0.5)
plt.xlabel('Temperature')
plt.ylabel('Demand')
plt.title('Temperature vs Demand')
plt.legend()
plt.show()

## Box plot

In [ ]:
plt.figure(figsize=(12, 6))
nsw_df[['FORECASTDEMAND', 'TOTALDEMAND', 'TEMPERATURE']].plot(kind='box')
plt.title('Distribution of Prices and Temperature')
plt.show()

## Histogram

In [ ]:
plt.figure(figsize=(14, 7))
nsw_df['TOTALDEMAND'].hist(alpha=0.7, label='Actual Price', bins=20)
nsw_df['FORECASTDEMAND'].hist(alpha=0.5, label='Forecast Price', bins=20)
plt.xlabel('Demand')
plt.ylabel('Frequency')
plt.title('Frequency Distribution of Demand')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
nsw_df['TEMPERATURE'].hist(color='orange', bins=20)
plt.xlabel('Temperature')
plt.ylabel('Frequency')
plt.title('Frequency Distribution of Temperature')
plt.show()

## Correlation 

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(nsw_df[['FORECASTDEMAND', 'TOTALDEMAND', 'TEMPERATURE']].corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

## seasonal decomposition (additive)
It's important to have a defined frequency (e.g., 'H' for hourly, 'D' for daily). Let's assume it's hourly data for this example

In [ ]:
nsw_df = nsw_df.asfreq('30min')

Now, choose the demand for electricity for decomposition

In [ ]:
ts = nsw_df['TOTALDEMAND']  # fix_me: missing values
ts.head()

In [ ]:
ts.isnull().sum()

In [ ]:
ts.isna().sum()

Perform seasonal decomposition. 
compare `model`=`multiplicative` vs `model`=`additive` if necessary

In [ ]:
decomposition = sm.tsa.seasonal_decompose(ts, model='additive')   
# fix_me: deal with missing values, temporarily using ffill

Plot the decomposed components of the time series

In [ ]:
fig = decomposition.plot()
plt.show()

## additive vs multiplicative relationships between components